In [23]:
key = '~/work/key'
user = 'ubuntu'

with open('ansible_hosts.txt', 'r') as f:
    hosts = [l.strip() for l in f.readlines()]
master = hosts[0]
hosts_name = 'ansible_hosts.yaml'
with open(hosts_name, 'w') as f:
    f.write(f'all:\n')
    f.write(f'  vars:\n')
    f.write(f'    dist_master_addr: {master}\n')
    f.write(f'    dist_world_size: {len(hosts)}\n')
    f.write(f'  hosts:\n')
    for i, h in enumerate(hosts):
        f.write(f'    {h}:\n')
        f.write(f'      ansible_host: {h}\n')
        f.write(f'      dist_rank: {i}\n')
hosts = hosts_name

key, hosts, master

('~/work/key', 'ansible_hosts.yaml', '176.99.135.42')

In [2]:
import os
os.environ['ANSIBLE_FORKS'] = '42'

In [38]:
!rm -vf ~/.ssh/known_hosts
!ansible --private-key {key} --ssh-common-args='-o StrictHostKeyChecking=accept-new' -i {hosts} -u {user} all -m shell -a 'uname -a; df /'

removed '/home/jovyan/.ssh/known_hosts'
176.99.135.42 | CHANGED | rc=0 >>
Linux data-srv-test 5.4.0-81-generic #91-Ubuntu SMP Thu Jul 15 19:09:17 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
Filesystem     1K-blocks    Used Available Use% Mounted on
/dev/vda1       60795672 1967280  58812008   4% /


In [4]:
!ansible --private-key {key} -i {hosts} -u {user} all -m debug -a 'var=dist_rank'

176.99.135.61 | SUCCESS => {
    "dist_rank": 0
}
176.99.135.97 | SUCCESS => {
    "dist_rank": 1
}
176.99.135.94 | SUCCESS => {
    "dist_rank": 3
}
176.99.135.121 | SUCCESS => {
    "dist_rank": 4
}
176.99.135.60 | SUCCESS => {
    "dist_rank": 2
}
176.99.135.115 | SUCCESS => {
    "dist_rank": 5
}
176.99.135.40 | SUCCESS => {
    "dist_rank": 6
}
176.99.135.38 | SUCCESS => {
    "dist_rank": 7
}


In [26]:
#!ansible --private-key {key} -i {hosts} -u {user} all -m script -a setup-ic.sh

---

In [39]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} data-srv.yaml


PLAY [all] *********************************************************************

TASK [setup-ic.sh] *************************************************************
changed: [176.99.135.42]

TASK [chmod /var/www/fileserver] ***********************************************
changed: [176.99.135.42]

TASK [download.sh] *************************************************************
changed: [176.99.135.42]

PLAY RECAP *********************************************************************
176.99.135.42              : ok=3    changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



In [30]:
#!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'rm -Rvf /kaggle /tmp/work'

In [5]:
os.environ['DATA_SRV'] = '176.99.135.45'
!ansible-playbook --private-key {key} -i {hosts} -u {user} data-srv-cli.yaml


PLAY [all] *********************************************************************

TASK [setup-ic.sh] *************************************************************
changed: [176.99.135.97]
changed: [176.99.135.38]
changed: [176.99.135.94]
changed: [176.99.135.115]
changed: [176.99.135.40]
changed: [176.99.135.121]
changed: [176.99.135.60]
changed: [176.99.135.61]

TASK [setup.sh] ****************************************************************
changed: [176.99.135.121]
changed: [176.99.135.94]
changed: [176.99.135.38]
changed: [176.99.135.97]
changed: [176.99.135.115]
changed: [176.99.135.40]
changed: [176.99.135.61]
changed: [176.99.135.60]

TASK [download.sh] *************************************************************
changed: [176.99.135.38]
changed: [176.99.135.94]
changed: [176.99.135.97]
changed: [176.99.135.121]
changed: [176.99.135.40]
changed: [176.99.135.115]
changed: [176.99.135.60]
changed: [176.99.135.61]

TASK [check /tmp/work] ******************************************

In [6]:
#!ansible-playbook --private-key {key} -i {hosts} -u {user} data.yaml

---

In [8]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} jupyter.yaml


PLAY [all] *********************************************************************

TASK [prepare] *****************************************************************
changed: [176.99.135.40]
changed: [176.99.135.115]
changed: [176.99.135.94]
changed: [176.99.135.121]
changed: [176.99.135.97]
changed: [176.99.135.38]
changed: [176.99.135.60]
changed: [176.99.135.61]

TASK [start] *******************************************************************
changed: [176.99.135.94]
changed: [176.99.135.61]
changed: [176.99.135.97]
changed: [176.99.135.60]
changed: [176.99.135.121]
changed: [176.99.135.115]
changed: [176.99.135.40]
changed: [176.99.135.38]

PLAY RECAP *********************************************************************
176.99.135.115             : ok=2    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
176.99.135.121             : ok=2    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
176.99.135.38              : ok=

In [9]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} jupyter-status.yaml


PLAY [all] *********************************************************************

TASK [check screen] ************************************************************
changed: [176.99.135.94]
changed: [176.99.135.38]
changed: [176.99.135.60]
changed: [176.99.135.115]
changed: [176.99.135.40]
changed: [176.99.135.97]
changed: [176.99.135.121]
changed: [176.99.135.61]

TASK [find url] ****************************************************************
changed: [176.99.135.94]
changed: [176.99.135.97]
changed: [176.99.135.121]
changed: [176.99.135.61]
changed: [176.99.135.40]
changed: [176.99.135.115]
changed: [176.99.135.38]
changed: [176.99.135.60]

TASK [url] *********************************************************************
ok: [176.99.135.61] => {
    "jupyter_url.stdout": "[I 2022-10-06 10:35:19.232 ServerApp]  or http://176.99.135.61:8888/lab?token=27e14725d61bd3c109b073ebbec2f361155c15bd46ed07ed"
}
ok: [176.99.135.97] => {
    "jupyter_url.stdout": "[I 2022-10-06 10:35:17.253 ServerA

In [7]:
#!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'cat /tmp/work/jupyter.log'

In [11]:
#!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls jupyter; ps -Af | grep jupyter'
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls jupyter'

176.99.135.94 | CHANGED | rc=0 >>
33mThere is a screen on:
	5487.jupyter	(10/06/2022 10:35:15 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.38 | CHANGED | rc=0 >>
33mThere is a screen on:
	5630.jupyter	(10/06/2022 10:35:15 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.121 | CHANGED | rc=0 >>
33mThere is a screen on:
	5537.jupyter	(10/06/2022 10:35:15 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.97 | CHANGED | rc=0 >>
33mThere is a screen on:
	5568.jupyter	(10/06/2022 10:35:15 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.40 | CHANGED | rc=0 >>
33mThere is a screen on:
	5553.jupyter	(10/06/2022 10:35:16 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.115 | CHANGED | rc=0 >>
33mThere is a screen on:
	5519.jupyter	(10/06/2022 10:35:15 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.61 | CHANGED | rc=0 >>
33mThere is a screen on:
	5379.jupyter	(10/06/2022 10:35:15 AM)	(Detached)
1 Socket in /run/screen/S-ubunt

---

In [17]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -S jupyter -X quit'

176.99.135.38 | CHANGED | rc=0 >>

176.99.135.40 | FAILED | rc=1 >>
31mThere is a screen on:
	4308.jupyter	(10/06/2022 05:26:20 AM)	(Dead ???)
31mRemove dead screens with 'screen -wipe'.
No screen session found.non-zero return code
176.99.135.11 | CHANGED | rc=0 >>

176.99.135.108 | CHANGED | rc=0 >>

176.99.135.115 | CHANGED | rc=0 >>

176.99.135.27 | CHANGED | rc=0 >>

176.99.135.42 | CHANGED | rc=0 >>

176.99.135.123 | CHANGED | rc=0 >>



In [18]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls; ps -Af|grep jupyter'

176.99.135.27 | CHANGED | rc=0 >>
No Sockets found in /run/screen/S-ubuntu.
33m
ubuntu      4597    4595  0 05:50 pts/0    00:00:00 /bin/sh -c screen -ls; ps -Af|grep jupyter
ubuntu      4600    4597  0 05:50 pts/0    00:00:00 grep jupyter
176.99.135.40 | CHANGED | rc=0 >>
33mThere is a screen on:
	4308.jupyter	(10/06/2022 05:26:20 AM)	(Dead ???)
33mRemove dead screens with 'screen -wipe'.
33m1 Socket in /run/screen/S-ubuntu.
ubuntu      4501    4499  0 05:50 pts/0    00:00:00 /bin/sh -c screen -ls; ps -Af|grep jupyter
ubuntu      4504    4501  0 05:50 pts/0    00:00:00 grep jupyter
176.99.135.115 | CHANGED | rc=0 >>
No Sockets found in /run/screen/S-ubuntu.
33m
ubuntu      4410    4408  0 05:50 pts/0    00:00:00 /bin/sh -c screen -ls; ps -Af|grep jupyter
ubuntu      4413    4410  0 05:50 pts/0    00:00:00 grep jupyter
176.99.135.42 | CHANGED | rc=0 >>
No Sockets found in /run/screen/S-ubuntu.
33m
ubuntu      4496    4494  0 05:50 pts/0    00:00:00 /bin/sh -c screen -ls; ps -Af|grep ju

In [34]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls jupyter'

176.99.135.27 | FAILED | rc=1 >>
No Sockets found in /run/screen/S-ubuntu.non-zero return code


---

In [21]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -S run -X quit; rm -f ~/work/*-log.txt ~/work/*_out.ipynb'

176.99.135.61 | CHANGED | rc=0 >>
No screen session found.
176.99.135.94 | CHANGED | rc=0 >>

176.99.135.40 | CHANGED | rc=0 >>

176.99.135.121 | CHANGED | rc=0 >>

176.99.135.97 | CHANGED | rc=0 >>

176.99.135.38 | CHANGED | rc=0 >>

176.99.135.60 | CHANGED | rc=0 >>

176.99.135.115 | CHANGED | rc=0 >>



In [22]:
#os.environ['RUN_NB'] = 'test.ipynb'
os.environ['RUN_NB'] = 'FaceId_multigpu_distributed.ipynb'
os.environ['RUN_WD'] = 'work'
!ansible-playbook --private-key {key} -i {hosts} -u {user} run.yaml


PLAY [all] *********************************************************************

TASK [copy] ********************************************************************
ok: [176.99.135.97] => (item=run.sh)
ok: [176.99.135.94] => (item=run.sh)
ok: [176.99.135.38] => (item=run.sh)
ok: [176.99.135.40] => (item=run.sh)
ok: [176.99.135.115] => (item=run.sh)
ok: [176.99.135.121] => (item=run.sh)
ok: [176.99.135.60] => (item=run.sh)
ok: [176.99.135.61] => (item=run.sh)
ok: [176.99.135.94] => (item=run.py)
ok: [176.99.135.97] => (item=run.py)
ok: [176.99.135.38] => (item=run.py)
ok: [176.99.135.121] => (item=run.py)
ok: [176.99.135.40] => (item=run.py)
ok: [176.99.135.115] => (item=run.py)
ok: [176.99.135.60] => (item=run.py)
ok: [176.99.135.61] => (item=run.py)
changed: [176.99.135.115] => (item=FaceId_multigpu_distributed.ipynb)
changed: [176.99.135.94] => (item=FaceId_multigpu_distributed.ipynb)
changed: [176.99.135.121] => (item=FaceId_multigpu_distributed.ipynb)
changed: [176.99.135.97] => (it

In [13]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls run'

176.99.135.94 | CHANGED | rc=0 >>
33mThere is a screen on:
	5704.run	(10/06/2022 10:36:33 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.121 | CHANGED | rc=0 >>
33mThere is a screen on:
	5754.run	(10/06/2022 10:36:33 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.38 | CHANGED | rc=0 >>
33mThere is a screen on:
	5855.run	(10/06/2022 10:36:33 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.97 | CHANGED | rc=0 >>
33mThere is a screen on:
	5785.run	(10/06/2022 10:36:33 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.115 | CHANGED | rc=0 >>
33mThere is a screen on:
	5736.run	(10/06/2022 10:36:33 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.60 | CHANGED | rc=0 >>
33mThere is a screen on:
	5446.run	(10/06/2022 10:36:34 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.40 | CHANGED | rc=0 >>
33mThere is a screen on:
	5772.run	(10/06/2022 10:36:33 AM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.61 | FAILED | 

In [16]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'tail ~/work/*-log.txt'

176.99.135.94 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceId_multigpu_distributed_out.ipynb
+ [ -z 1 ]
+ python run.py FaceId_multigpu_distributed.ipynb FaceId_multigpu_distributed_out.ipynb .
[IPKernelApp] WARNING | Parent appears to have exited, shutting down.
176.99.135.115 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceId_multigpu_distributed_out.ipynb
+ [ -z 1 ]
+ python run.py FaceId_multigpu_distributed.ipynb FaceId_multigpu_distributed_out.ipynb .
[IPKernelApp] WARNING | Parent appears to have exited, shutting down.
176.99.135.97 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceId_multigpu_distributed_out.ipynb
+ [ -z 1 ]
+ python run.py FaceId_multigpu_distributed.ipynb FaceId_multigpu_distributed_out.ipynb .
[IPKernelApp] WARNING | Parent appears to have exited, shutting down.
176.99.135.38 | CHANGED | rc=0 >>
+ out=
+ [ 

In [17]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'rm -f ~/work/*-log.txt ~/work/*_out.ipynb; ls -l ~/work/'

176.99.135.94 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.40 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.60 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.115 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.61 | CHANGED | rc=0 >>
total 100
-rwxr-xr-x 1 ubuntu ubu

In [15]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -S run -X quit'

176.99.135.94 | CHANGED | rc=0 >>

176.99.135.60 | CHANGED | rc=0 >>

176.99.135.61 | FAILED | rc=1 >>
No screen session found.non-zero return code
176.99.135.121 | CHANGED | rc=0 >>

176.99.135.115 | CHANGED | rc=0 >>

176.99.135.40 | CHANGED | rc=0 >>

176.99.135.97 | CHANGED | rc=0 >>

176.99.135.38 | CHANGED | rc=0 >>



---